# 날씨 데이터 크롤링

출처: https://www.data.go.kr/  
ASOS 종관기상관측 데이터 활용

# URL 설정

# 시간별자료

In [1]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
key='13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D'
url="http://data.kma.go.kr/apiData/getData"


### api 설명서에 있는 활용가이드에서 요청/응답 메세지 예제를 확인한다

### urlparse, parse_qs를 통해 파라미터로 나눠준다

In [12]:
from urllib.parse import urlparse, parse_qs
url = urlparse("http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=HR&stnIds=108&schListCnt=10&endDt=20200101&endHh=01&startHh=01&startDt=20100101&dataType=JSON")
par_dic = parse_qs(a.query)
for key in par_dic:
    par_dic[key] = par_dic[key][0]
par_dic

{'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
 'numOfRows': '10',
 'pageNo': '1',
 'dataCd': 'ASOS',
 'dateCd': 'HR',
 'stnIds': '108',
 'schListCnt': '10',
 'endDt': '20200101',
 'endHh': '01',
 'startHh': '01',
 'startDt': '20100101',
 'dataType': 'JSON'}

In [135]:
def get_hourly_data(start_date, start_hour,end_date,end_hour):
    start_date=start_date
    start_hour=start_hour
    end_date=end_date
    end_hour=end_hour
    url='http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

    
    params={'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
     'numOfRows': '10',
     'pageNo': '1',
     'dataCd': 'ASOS',
     'dateCd': 'HR',
     'stnIds': '108',
     'schListCnt': '10',
     'endDt': end_date,
     'endHh': end_hour,
     'startHh': start_hour,
     'startDt': start_date,
     'dataType': 'JSON'
           }

    resp = requests.get(url, params=params)
    #result= json.loads(resp.text)

    return resp.text

In [136]:
data=get_hourly_data('20200605','08','20200605','09')
json.loads(data)

{'response': {'header': {'resultCode': '99',
   'resultMsg': '전날 자료까지 제공됩니다. 날짜 범위를 확인해주세요.'}}}

# 일별데이터 출력

In [56]:
from urllib.parse import urlparse, parse_qs
url = urlparse("http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=DAY&startDt=20100101&endDt=20100102&stnIds=108")
par_dic = parse_qs(url.query)
for key in par_dic:
    par_dic[key] = par_dic[key][0]
par_dic

{'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
 'numOfRows': '10',
 'pageNo': '1',
 'dataCd': 'ASOS',
 'dateCd': 'DAY',
 'startDt': '20100101',
 'endDt': '20100102',
 'stnIds': '108'}

In [59]:
def get_daily_data(start_date,end_date):
    start_date=start_date
    end_date=end_date
    url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

    
    params={'serviceKey': unquote('13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D'),
            'numOfRows': '200',
            'pageNo': '1',
            'dataCd': 'ASOS',
            'dateCd': 'DAY',
            'startDt': start_date,
            'endDt': end_date,
            'stnIds': '108',
            'dataType':'JSON'
           }

    resp = requests.get(url, params=params)
    result= json.loads(resp.text)
    return result

In [60]:
def data_to_DataFrame(result):
    total_data=result['response']['body']['items']['item']
    
    result = {'날짜':[],
              '최저기온':[],
              '최고기온':[],
              '비':[],
              '강수량':[],
             }
    for data in total_data:
        result['날짜'].append(data['tm'])
        result['최저기온'].append(data['minTa'])
        result['최고기온'].append(data['maxTa'])
        if data['sumRn']>0:
            result['비'].append('O')
        else:
            result['비'].append('X')
        result['강수량'].append(data['sumRn'])
        
    df=pd.DataFrame(result)
    
    return df

In [63]:
def append_today(data):
    url = 'https://www.weatheri.co.kr/forecast/forecast01.php'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, from_encoding='utf-8')
    attrs={'width':'615',
      'border':'0',
      'cellpadding':'1',
      'cellspacing':'1',
       'bgcolor':'#BCBFC2'
      }
    inner_table = soup.find('table',attrs=attrs)
    date = inner_table.select('td b')[0].text
    date = '2020-{0}-{1}'.format(date.split()[0][:2],date.split()[1][:2])
    high = inner_table.select('table tr td b font')[0].text
    high = high.replace('˚C','')
    low = inner_table.select('table tr td b font')[0].text
    low = low.replace('˚C','')
    sum_rain=inner_table.select('table tr td font')[-1].text
    if sum_rain !='- mm':
        rain='O',
        sum_rain.split()[1][:-2]
        
    else:
        rain='X'
        sum_rain = 0
    
    new_data=pd.DataFrame({'날짜':[date],
             '최저기온':[low],
             '최고기온':[high],
             '비':[rain],
             '강수량':[sum_rain]})
    
    data = data.append(new_data,ignore_index=True)
    print(data)
    return data

In [64]:
result=get_daily_data('20200101','20200606')
output=data_to_DataFrame(result)
d=append_today(output)

             날짜  최저기온  최고기온  비  강수량
0    2020-01-01  -6.5   0.3  O  0.1
1    2020-01-02  -0.7   3.8  X  0.0
2    2020-01-03  -3.4   4.6  X  0.0
3    2020-01-04  -2.8   6.1  X  0.0
4    2020-01-05  -3.2   6.6  X  0.0
..          ...   ...   ... ..  ...
154  2020-06-03  17.6  28.4  X  0.0
155  2020-06-04  19.7  26.5  O  0.2
156  2020-06-05  18.4  28.7  X  0.0
157  2020-06-06  19.2  30.9  X  0.0
158  2020-06-07    28    28  X  0.0

[159 rows x 5 columns]


In [ ]:
d

In [65]:
d.to_excel('weather_crawling.xlsx')